# **Applicatons of RNNs - Natural Language Prediction**
 * # We now build a  Recurrent Neural Network in Keras  to predict next character in the the Arthur Conan Doyle's book "The Adventures of Sherlock Holmes"

In [ ]:
import numpy as np
import keras
from keras.models import Sequential, load_model
from keras.layers import LSTM, GRU, Dense, Input, Bidirectional
#from keras.layers.core import  Activation
from keras.callbacks import EarlyStopping, ModelCheckpoint
import tensorflow as tf

In [ ]:
!curl -O https://www.gutenberg.org/files/1661/1661-0.txt

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  593k  100  593k    0     0   490k      0  0:00:01  0:00:01 --:--:--  490k


In [ ]:
path = '1661-0.txt'
text = open(path).read().lower()
print('corpus length:', len(text))
text = text.replace("\n", " ")  # We remove newlines chars for nicer display
print(f"text[:100]:{text[:100]}")
chars = sorted(list(set(text)))
print("Total distinct chars:", len(chars))
# char dictionaries
char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))


corpus length: 581425
text[:100]:﻿the project gutenberg ebook of the adventures of sherlock holmes, by arthur conan doyle  this ebook
Total distinct chars: 71


In [ ]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)

Mounted at /content/gdrive


In [ ]:
%cd gdrive/MyDrive/MLNDNN/Notebooks/Lecture13_2025/data

/content/gdrive/MyDrive/INSEGNAMENTI/MLDNN/MLDNN_course/Module2_course/data
1661-0.txt


# **Create sequences/sentencies of 40 characters and predict the next one**

In [ ]:
# cut the text in overlapping sequences of window_size characters
# each time we slide of one character
window_size = 40

sentences = []
next_chars = []
for i in range(len(text) - window_size):
    sentences.append(text[i : i + window_size])
    next_chars.append(text[i + window_size])

print("Number of sequences:", len(sentences))

Number of sequences: 581385


# **Character encoding: one-hot**
 * # Again, better and more efficient encoding can be defined
 * # Here we do not focus on performance

In [ ]:
# Creating one hot encoding
x = np.zeros((len(sentences), window_size, len(chars)), dtype=bool)
y = np.zeros((len(sentences), len(chars)), dtype=bool)
for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        x[i, t, char_indices[char]] = 1
    y[i, char_indices[next_chars[i]]] = 1


In [ ]:
print(x.shape, y.shape)

(581385, 40, 71) (581385, 71)


# **Build the model: a single LSTM layer**

In [ ]:
from keras.models import load_model
model_file = 'NCP_model.keras'
import os
if os.path.isfile(model_file):
  print("file exists, loading")
  model = load_model(model_file)
else:
  model = keras.Sequential(
    [
        Input(shape=(window_size, len(chars))),
        LSTM(128),
        Dense(len(chars), activation="softmax"),
    ]
  )

print(model.summary())

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ lstm (LSTM)                     │ (None, 128)            │       102,400 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 71)             │         9,159 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 111,559 (435.78 KB)

 Trainable params: 111,559 (435.78 KB)

 Non-trainable params: 0 (0.00 B)

None


# **Going deeper: multiple LSTM hidden layers**
 * # Need to return output sequences from previous layers
* # can be done by setting argument ```return_sequences=True```  

In [ ]:
import os
model_file2 = 'NCP2_model.keras'

if os.path.isfile(model_file2):
  print("file exists, loading")
  model2 = load_model(model_file)
else:
  model2 = keras.Sequential(
    [
        Input(shape=(window_size, len(chars))),
        LSTM(82, return_sequences=True),# deep RNN
        LSTM(82),
        Dense(len(chars), activation="softmax")
    ]
  )

print(model2.summary())

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ lstm_1 (LSTM)                   │ (None, 40, 82)         │        50,512 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_2 (LSTM)                   │ (None, 82)             │        54,120 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 71)             │         5,893 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 110,525 (431.74 KB)

 Trainable params: 110,525 (431.74 KB)

 Non-trainable params: 0 (0.00 B)

None


# **Constructing bidirectional RNN**
 * # In keras can be done by using the ```Bidirectional``` layer

In [ ]:
model_file3 = 'NCP3_model.keras'
if os.path.isfile(model_file3):
  print("file exists, loading")
  model3 = load_model(model_file)
else:
  model3 = keras.Sequential(
    [
        Input(shape=(window_size, len(chars))),
        Bidirectional(LSTM(82)),
        Dense(len(chars), activation="softmax"),
    ]
  )

print(model3.summary())

Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ bidirectional (Bidirectional)   │ (None, 164)            │       101,024 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 71)             │        11,715 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 112,739 (440.39 KB)

 Trainable params: 112,739 (440.39 KB)

 Non-trainable params: 0 (0.00 B)

None


In [ ]:
model_file4 = 'NCP4_model.keras'

if os.path.isfile(model_file4):
  print("file exists, loading")
  model4 = load_model(model_file)
else:
  model4 = keras.Sequential(
    [
        Input(shape=(window_size, len(chars))),
        GRU(150),
        Dense(len(chars), activation="softmax"),
    ]
  )

print(model4.summary())

Model: "sequential_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ gru (GRU)                       │ (None, 150)            │       100,350 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 71)             │        10,721 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 111,071 (433.87 KB)

 Trainable params: 111,071 (433.87 KB)

 Non-trainable params: 0 (0.00 B)

None



# **Training the models**

In [ ]:
if not os.path.isfile(model_file):
  checkpoint = ModelCheckpoint(model_file, monitor='val_loss', verbose=0,
                             save_best_only=True, mode='min')

  callbacks_list = [checkpoint]
  model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=10**-3),
                loss='categorical_crossentropy',
                metrics=['accuracy'])
  epochs = 10
  batch_size = 512
  model.fit(x, y,
            batch_size=batch_size,
            epochs=epochs,
            shuffle=True,
            callbacks=callbacks_list,
            validation_split=0.1)
#model.save_weights(model_file)

Epoch 1/10
1022/1022 ━━━━━━━━━━━━━━━━━━━━ 14s 11ms/step - accuracy: 0.2432 - loss: 2.7613 - val_accuracy: 0.3548 - val_loss: 2.2788
Epoch 2/10
1022/1022 ━━━━━━━━━━━━━━━━━━━━ 11s 11ms/step - accuracy: 0.3787 - loss: 2.1255 - val_accuracy: 0.3765 - val_loss: 2.1534
Epoch 3/10
1022/1022 ━━━━━━━━━━━━━━━━━━━━ 11s 11ms/step - accuracy: 0.4071 - loss: 1.9954 - val_accuracy: 0.4065 - val_loss: 2.0742
Epoch 4/10
1022/1022 ━━━━━━━━━━━━━━━━━━━━ 21s 11ms/step - accuracy: 0.4327 - loss: 1.9100 - val_accuracy: 0.4252 - val_loss: 2.0163
Epoch 5/10
1022/1022 ━━━━━━━━━━━━━━━━━━━━ 11s 10ms/step - accuracy: 0.4498 - loss: 1.8497 - val_accuracy: 0.4336 - val_loss: 1.9714
Epoch 6/10
1022/1022 ━━━━━━━━━━━━━━━━━━━━ 21s 11ms/step - accuracy: 0.4648 - loss: 1.7956 - val_accuracy: 0.4429 - val_loss: 1.9296
Epoch 7/10
1022/1022 ━━━━━━━━━━━━━━━━━━━━ 20s 11ms/step - accuracy: 0.4757 - loss: 1.7528 - val_accuracy: 0.4560 - val_loss: 1.8876
Epoch 8/10
1022/1022 ━━━━━━━━━━━━━━━━━━━━ 20s 11ms/step - accuracy: 0.4863 -

In [ ]:
if not os.path.isfile(model_file2):
  checkpoint = ModelCheckpoint(model_file2, monitor='val_loss', verbose=0,
                             save_best_only=True, mode='min')

  callbacks_list = [checkpoint]
  model2.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=10**-3),
                loss='categorical_crossentropy',
                metrics=['accuracy'])
  epochs = 10
  batch_size = 512
  model2.fit(x, y,
            batch_size=batch_size,
            epochs=epochs,
            shuffle=True,callbacks=callbacks_list,
            validation_split=0.1)
#model2.save_weights(model_file2)

Epoch 1/10
1022/1022 ━━━━━━━━━━━━━━━━━━━━ 17s 15ms/step - accuracy: 0.2107 - loss: 2.8992 - val_accuracy: 0.3241 - val_loss: 2.3778
Epoch 2/10
1022/1022 ━━━━━━━━━━━━━━━━━━━━ 15s 15ms/step - accuracy: 0.3471 - loss: 2.2272 - val_accuracy: 0.3680 - val_loss: 2.2302
Epoch 3/10
1022/1022 ━━━━━━━━━━━━━━━━━━━━ 20s 15ms/step - accuracy: 0.3876 - loss: 2.0786 - val_accuracy: 0.3874 - val_loss: 2.1376
Epoch 4/10
1022/1022 ━━━━━━━━━━━━━━━━━━━━ 20s 14ms/step - accuracy: 0.4121 - loss: 1.9837 - val_accuracy: 0.4016 - val_loss: 2.0762
Epoch 5/10
1022/1022 ━━━━━━━━━━━━━━━━━━━━ 15s 15ms/step - accuracy: 0.4270 - loss: 1.9218 - val_accuracy: 0.4142 - val_loss: 2.0335
Epoch 6/10
1022/1022 ━━━━━━━━━━━━━━━━━━━━ 15s 15ms/step - accuracy: 0.4426 - loss: 1.8691 - val_accuracy: 0.4346 - val_loss: 1.9832
Epoch 7/10
1022/1022 ━━━━━━━━━━━━━━━━━━━━ 20s 14ms/step - accuracy: 0.4552 - loss: 1.8276 - val_accuracy: 0.4381 - val_loss: 1.9559
Epoch 8/10
1022/1022 ━━━━━━━━━━━━━━━━━━━━ 21s 15ms/step - accuracy: 0.4658 -

In [ ]:
if not os.path.isfile(model_file3):
  checkpoint = ModelCheckpoint(model_file3, monitor='val_loss', verbose=0,
                             save_best_only=True, mode='min')

  callbacks_list = [checkpoint]
  checkpoint = ModelCheckpoint(model_file3, monitor='val_loss', verbose=0,
                             save_best_only=True, mode='min')

  callbacks_list = [checkpoint]
  model3.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=10**-3),
                loss='categorical_crossentropy',
                metrics=['accuracy'])
  epochs = 10
  batch_size = 512
  model3.fit(x, y,
            batch_size=batch_size,
            epochs=epochs,
            shuffle=True,callbacks=callbacks_list,
            validation_split=0.1)
#model3.save_weights(model_file3)

Epoch 1/10
1022/1022 ━━━━━━━━━━━━━━━━━━━━ 19s 16ms/step - accuracy: 0.2388 - loss: 2.7956 - val_accuracy: 0.3529 - val_loss: 2.2956
Epoch 2/10
1022/1022 ━━━━━━━━━━━━━━━━━━━━ 19s 15ms/step - accuracy: 0.3791 - loss: 2.1331 - val_accuracy: 0.3864 - val_loss: 2.1461
Epoch 3/10
1022/1022 ━━━━━━━━━━━━━━━━━━━━ 21s 15ms/step - accuracy: 0.4155 - loss: 1.9837 - val_accuracy: 0.4095 - val_loss: 2.0674
Epoch 4/10
1022/1022 ━━━━━━━━━━━━━━━━━━━━ 20s 15ms/step - accuracy: 0.4407 - loss: 1.8924 - val_accuracy: 0.4258 - val_loss: 2.0056
Epoch 5/10
1022/1022 ━━━━━━━━━━━━━━━━━━━━ 21s 16ms/step - accuracy: 0.4561 - loss: 1.8318 - val_accuracy: 0.4341 - val_loss: 1.9675
Epoch 6/10
1022/1022 ━━━━━━━━━━━━━━━━━━━━ 16s 15ms/step - accuracy: 0.4685 - loss: 1.7836 - val_accuracy: 0.4443 - val_loss: 1.9267
Epoch 7/10
1022/1022 ━━━━━━━━━━━━━━━━━━━━ 16s 15ms/step - accuracy: 0.4786 - loss: 1.7390 - val_accuracy: 0.4522 - val_loss: 1.8958
Epoch 8/10
1022/1022 ━━━━━━━━━━━━━━━━━━━━ 20s 15ms/step - accuracy: 0.4882 -

In [ ]:
if not os.path.isfile(model_file4):
  checkpoint = ModelCheckpoint(model_file4, monitor='val_loss', verbose=0,
                             save_best_only=True, mode='min')

  callbacks_list = [checkpoint]
  model4.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=10**-3),
                loss='categorical_crossentropy',
                metrics=['accuracy'])
  epochs = 10
  batch_size = 256
  model4.fit(x, y,
            batch_size=batch_size,
            epochs=epochs,
            shuffle=True,callbacks=callbacks_list,
            validation_split=0.1)
#model4.save_weights(model_file4)

Epoch 1/10
2044/2044 ━━━━━━━━━━━━━━━━━━━━ 18s 8ms/step - accuracy: 0.3101 - loss: 2.4707 - val_accuracy: 0.4269 - val_loss: 1.9870
Epoch 2/10
2044/2044 ━━━━━━━━━━━━━━━━━━━━ 19s 8ms/step - accuracy: 0.4687 - loss: 1.7767 - val_accuracy: 0.4759 - val_loss: 1.8100
Epoch 3/10
2044/2044 ━━━━━━━━━━━━━━━━━━━━ 21s 8ms/step - accuracy: 0.5207 - loss: 1.5980 - val_accuracy: 0.5019 - val_loss: 1.7249
Epoch 4/10
2044/2044 ━━━━━━━━━━━━━━━━━━━━ 21s 8ms/step - accuracy: 0.5468 - loss: 1.5024 - val_accuracy: 0.5190 - val_loss: 1.6777
Epoch 5/10
2044/2044 ━━━━━━━━━━━━━━━━━━━━ 16s 8ms/step - accuracy: 0.5643 - loss: 1.4362 - val_accuracy: 0.5255 - val_loss: 1.6510
Epoch 6/10
2044/2044 ━━━━━━━━━━━━━━━━━━━━ 20s 8ms/step - accuracy: 0.5773 - loss: 1.3893 - val_accuracy: 0.5291 - val_loss: 1.6352
Epoch 7/10
2044/2044 ━━━━━━━━━━━━━━━━━━━━ 16s 8ms/step - accuracy: 0.5859 - loss: 1.3549 - val_accuracy: 0.5379 - val_loss: 1.6116
Epoch 8/10
2044/2044 ━━━━━━━━━━━━━━━━━━━━ 20s 8ms/step - accuracy: 0.5946 - loss: 1

# Function to sample next character

In [ ]:
# given the predictite probability distribution pred, we need to sample the
#   the next character
#  We use a scheme used in thermodynamics named TEMPERATURE SAMPLING
# where high temperature means low energy states are more likely encountered.
# In probability models, logits play the role of energy and we can implement
# temperature sampling by dividing logits by the temperature before feeding
#    them into softmax and obtaining our sampling probabilities

def sample(preds, temperature=1.0):
    # helper function to sample an index from a probability array
    preds = np.asarray(preds).astype("float64")
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    # The multinomial distribution is a multivariate generalization of
    #   the binomial distribution. Take an experiment with one of p possible
    #    outcomes
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

# Regenerating the text

In [ ]:
# number of characters to generate
to_be_gen = 50
temperature = 1
generated = ''
print(f"text:{text[0:to_be_gen+window_size]}")
for i in range(to_be_gen):
    start_index = i
    sentence = text[start_index : start_index + window_size]

    x_pred = np.zeros((1, window_size, len(chars)))
    for t, char in enumerate(sentence):
        x_pred[0, t, char_indices[char]] = 1.0
    preds = model.predict(x_pred, verbose=0)[0]
#    next_index = sample(preds, temperature)
    next_index = np.argmax(preds)
#    next_word = list(word_dict.keys())[list(word_dict.values()).index(next_index)]
    next_char = indices_char[next_index]
    print(f"input: {sentence}, \t pred:{next_char}")
    sentence = sentence[1:] + next_char
    generated += next_char

print("...Generated: ", generated)


text:﻿the project gutenberg ebook of the adventures of sherlock holmes, by arthur conan doyle  
input: ﻿the project gutenberg ebook of the adve, 	 pred:r
input: the project gutenberg ebook of the adven, 	 pred:t
input: he project gutenberg ebook of the advent, 	 pred:i
input: e project gutenberg ebook of the adventu, 	 pred:r
input:  project gutenberg ebook of the adventur, 	 pred:e
input: project gutenberg ebook of the adventure, 	 pred: 
input: roject gutenberg ebook of the adventures, 	 pred: 
input: oject gutenberg ebook of the adventures , 	 pred:a
input: ject gutenberg ebook of the adventures o, 	 pred:f
input: ect gutenberg ebook of the adventures of, 	 pred: 
input: ct gutenberg ebook of the adventures of , 	 pred:t
input: t gutenberg ebook of the adventures of s, 	 pred:h
input:  gutenberg ebook of the adventures of sh, 	 pred:e
input: gutenberg ebook of the adventures of she, 	 pred: 
input: utenberg ebook of the adventures of sher, 	 pred:e
input: tenberg ebook of the advent

# **Exercise**
 * # choose a model among those above and evaluate what is better between:
* # having smaller states but more hidden layers
* # having larger states but less hidden layers
* # almost equal the number of parameters
* # **Increase the number of epochs at least to 20/30**  

In [ ]:
drive.flush_and_unmount()